## Classifier - Try 3

Classify articles frame using SRL and a classifier

In [11]:
!pip install allennlp allennlp-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 109.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 110.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 91.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 65.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 95.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [83]:
import os

try:
  import google.colab

  from google.colab import drive
  drive.mount('/content/drive')
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  os.chdir('drive/MyDrive/Git/MasterThesis/data')
else:
  os.chdir('../../data/')

labels_path = "data/en/train-labels-subtask-2.txt"
articles_path = "data/en/train-articles-subtask-2/"

FileNotFoundError: [Errno 2] No such file or directory: '../../data/'

In [84]:
import pandas as pd

# Read the dev-labels-subtask-2.txt file
labels_df = pd.read_csv(labels_path, sep="\t")

# Rename the columns for easier processing
labels_df.columns = ["article_id", "frames"]


labels_df.head()

,article_id,frames
0,832959523,"Morality,Security_and_defense,Policy_prescript..."
1,833039623,"Political,Crime_and_punishment,External_regula..."
2,833032367,"Political,Crime_and_punishment,Fairness_and_eq..."
3,814777937,"Political,Morality,Fairness_and_equality,Exter..."
4,821744708,"Policy_prescription_and_evaluation,Political,L..."


In [85]:
# A function to read the article text given its ID
def get_article_content(article_id):
    try:
        with open(f"{articles_path}/article{article_id}.txt", "r") as f:
            return f.read()
    except FileNotFoundError:
        return None

df = labels_df

# Apply the function to get the article content
df["content"] = df["article_id"].apply(get_article_content)

# Drop rows where content could not be found
df.dropna(subset=["content"], inplace=True)

df.head()


,article_id,frames,content
0,832959523,"Morality,Security_and_defense,Policy_prescript...",How Theresa May Botched\n\nThose were the time...
1,833039623,"Political,Crime_and_punishment,External_regula...",Robert Mueller III Rests His Case—Dems NEVER W...
2,833032367,"Political,Crime_and_punishment,Fairness_and_eq...",Robert Mueller Not Recommending Any More Indic...
3,814777937,"Political,Morality,Fairness_and_equality,Exter...",The Far Right Is Trying to Co-opt the Yellow V...
4,821744708,"Policy_prescription_and_evaluation,Political,L...",‘Special place in hell’ for those who promoted...


In [86]:
# Split the frames column into a list of frames
df["frames_list"] = df["frames"].str.split(",")

# create for each frame a new column with the frame as name and 1 if the frame is present in the article and 0 if not
for frame in df["frames_list"].explode().unique():
    df[frame] = df["frames_list"].apply(lambda x: 1 if frame in x else 0)

df.head()

,article_id,frames,content,frames_list,Morality,Security_and_defense,Policy_prescription_and_evaluation,Legality_Constitutionality_and_jurisprudence,Economic,Political,Crime_and_punishment,External_regulation_and_reputation,Public_opinion,Fairness_and_equality,Capacity_and_resources,Quality_of_life,Cultural_identity,Health_and_safety
0,832959523,"Morality,Security_and_defense,Policy_prescript...",How Theresa May Botched\n\nThose were the time...,"[Morality, Security_and_defense, Policy_prescr...",1,1,1,1,1,0,0,0,0,0,0,0,0,0
1,833039623,"Political,Crime_and_punishment,External_regula...",Robert Mueller III Rests His Case—Dems NEVER W...,"[Political, Crime_and_punishment, External_reg...",0,0,1,1,0,1,1,1,1,0,0,0,0,0
2,833032367,"Political,Crime_and_punishment,Fairness_and_eq...",Robert Mueller Not Recommending Any More Indic...,"[Political, Crime_and_punishment, Fairness_and...",0,0,0,1,0,1,1,1,0,1,0,0,0,0
3,814777937,"Political,Morality,Fairness_and_equality,Exter...",The Far Right Is Trying to Co-opt the Yellow V...,"[Political, Morality, Fairness_and_equality, E...",1,1,0,0,1,1,0,1,1,1,0,0,0,0
4,821744708,"Policy_prescription_and_evaluation,Political,L...",‘Special place in hell’ for those who promoted...,"[Policy_prescription_and_evaluation, Political...",0,0,1,1,0,1,0,1,0,0,0,0,0,0


In [87]:
X = df["content"]
y = df.drop(columns=["article_id", "frames", "frames_list", "content"])

In [88]:
X.head()

0    How Theresa May Botched\n\nThose were the time...
1    Robert Mueller III Rests His Case—Dems NEVER W...
2    Robert Mueller Not Recommending Any More Indic...
3    The Far Right Is Trying to Co-opt the Yellow V...
4    ‘Special place in hell’ for those who promoted...
Name: content, dtype: object

In [89]:
y.head()

,Morality,Security_and_defense,Policy_prescription_and_evaluation,Legality_Constitutionality_and_jurisprudence,Economic,Political,Crime_and_punishment,External_regulation_and_reputation,Public_opinion,Fairness_and_equality,Capacity_and_resources,Quality_of_life,Cultural_identity,Health_and_safety
0,1,1,1,1,1,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,1,1,1,1,0,0,0,0,0
2,0,0,0,1,0,1,1,1,0,1,0,0,0,0
3,1,1,0,0,1,1,0,1,1,1,0,0,0,0
4,0,0,1,1,0,1,0,1,0,0,0,0,0,0


In [90]:
len(X), len(y)

(432, 432)

### Create Dataset

In [91]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')

### Extract SRL Embeddings from articles

In [47]:
!pip install pycuda

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 28.8 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp39-cp39-linux_x86_64.whl size=661947 sha256=ccc879f0eafff21d0271d2a2c657b5f4760a433a25086155ac5f9f1ff793a293
  Stored in directory: /root/.cache/pip/wheels/c8/24/b2/c54ef2a7a3dcb9daa9e403c036360b7e47a8200c94413d0c77
Successfully built pycuda


In [92]:
import pycuda
from pycuda import compiler
import pycuda.driver as drv

drv.init()
print("%d device(s) found." % drv.Device.count())
           
for ordinal in range(drv.Device.count()):
    dev = drv.Device(ordinal)
    print (ordinal, dev.name())

1 device(s) found.
0 NVIDIA RTX A4000


In [93]:
from allennlp.predictors.predictor import Predictor
from allennlp_models.structured_prediction.models import srl_bert

In [94]:
# Load the SRL predictor
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz", cuda_device=0)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [95]:
from nltk.tokenize import sent_tokenize
import pandas as pd

In [96]:
def batched_extract_srl_components(sentences, predictor):
    # Prepare the batched input for the predictor
    batched_input = [{'sentence': sentence} for sentence in sentences]
    batched_srl = predictor.predict_batch_json(batched_input)
    
    # Extract SRL components from the batched predictions
    results = []
    for srl in batched_srl:
        best_extracted_data = None
        second_best_extracted_data = None
        for verb_entry in srl['verbs']:
            tags = verb_entry['tags']
            arg0_indices = [i for i, tag in enumerate(tags) if tag in ['B-ARG0', 'I-ARG0']]
            arg1_indices = [i for i, tag in enumerate(tags) if tag in ['B-ARG1', 'I-ARG1']]

            if arg0_indices and arg1_indices:
                best_extracted_data = {
                    'predicate': verb_entry['verb'],
                    'ARG0': ' '.join([srl['words'][i] for i in arg0_indices]),
                    'ARG1': ' '.join([srl['words'][i] for i in arg1_indices])
                }
                break
            elif (arg0_indices or arg1_indices) and not second_best_extracted_data:
                second_best_extracted_data = {
                    'predicate': verb_entry['verb'],
                    'ARG0': ' '.join([srl['words'][i] for i in arg0_indices]) if arg0_indices else '',
                    'ARG1': ' '.join([srl['words'][i] for i in arg1_indices]) if arg1_indices else ''
                }

        if best_extracted_data:
            results.append(best_extracted_data)
        elif second_best_extracted_data:
            results.append(second_best_extracted_data)
            
    return results

def optimized_extract_srl(X, predictor, batch_size=32):
    total_articles = len(X)
    processed_articles = 0

    all_results = []

    for article in X:
        sentences = sent_tokenize(article)
        article_srls = []

        for i in range(0, len(sentences), batch_size):
            batched_sentences = sentences[i:i+batch_size]
            article_srls.extend(batched_extract_srl_components(batched_sentences, predictor))

        all_results.append(article_srls)
        processed_articles += 1
        print(f"Processed article {processed_articles}/{total_articles}")

    return pd.Series(all_results)

In [97]:
X_srl = optimized_extract_srl(X, predictor)

Processed article 1/432
Processed article 2/432
Processed article 3/432
Processed article 4/432
Processed article 5/432
Processed article 6/432
Processed article 7/432
Processed article 8/432
Processed article 9/432
Processed article 10/432
Processed article 11/432
Processed article 12/432
Processed article 13/432
Processed article 14/432
Processed article 15/432
Processed article 16/432
Processed article 17/432
Processed article 18/432
Processed article 19/432
Processed article 20/432
Processed article 21/432
Processed article 22/432
Processed article 23/432
Processed article 24/432
Processed article 25/432
Processed article 26/432
Processed article 27/432
Processed article 28/432
Processed article 29/432
Processed article 30/432
Processed article 31/432
Processed article 32/432
Processed article 33/432
Processed article 34/432
Processed article 35/432
Processed article 36/432
Processed article 37/432
Processed article 38/432
Processed article 39/432
Processed article 40/432
Processed

In [98]:
X_srl[0]

[{'predicate': 'were', 'ARG0': '', 'ARG1': 'Those'},
 {'predicate': 'beginning',
  'ARG0': '',
  'ARG1': 'Negotiations between the United Kingdom and the European Union about Brexit'},
 {'predicate': 'characterizes',
  'ARG0': 'The “ you ’ll be crushed ” arrogance in the headline',
  'ARG1': 'the attitude the British government under May demonstrated during the talks'},
 {'predicate': 'changed', 'ARG0': '', 'ARG1': 'that attitude'},
 {'predicate': 'writes',
  'ARG0': 'The BBC',
  'ARG1': 'Against the State : An ... Llewellyn H. Rockwell Jr. Best Price : $ 12.99 Buy New $ 9.05 ( as of 10:25 EDT - Details ) Theresa May has said she “ sincerely hopes ” the UK will leave the EU with a deal and she is still “ working on ” ensuring Parliament ’s agreement . Arriving in Brussels , she said that she had “ personal regret ” over her request to delay Brexit'},
 {'predicate': 'try',
  'ARG0': 'the PM',
  'ARG1': 'to get their backing for a delay beyond 29 March'},
 {'predicate': 'said',
  'ARG0':

In [108]:
X_srl.to_pickle("../notebooks/classifier/X_srl.pkl")

# Dataset

In [109]:
import torch

# Dataset
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW

In [133]:
def get_embedding(sentence, tokenizer, max_length=512):
    return tokenizer.encode_plus(
                sentence,
                add_special_tokens=True,
                max_length=max_length,
                return_token_type_ids=False,
                padding='max_length',
                return_attention_mask=True,
                truncation=True
            )

class ArticleDataset(Dataset):
    def __init__(self, texts, srl_data, labels, tokenizer, max_length=512):
        self.texts = texts
        self.srl_data = srl_data
        self.labels = labels.values
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        article = self.texts.iloc[idx]
        sentences = sent_tokenize(article)
        srl_list = self.srl_data.iloc[idx]  # Assuming this is a list of SRL dicts for each sentence
        label = self.labels[idx]

        sentence_encodings = []
        srl_encodings = []

        for sentence, srl in zip(sentences, srl_list):
            # Encode the sentence
            sentence_encoding = get_embedding(sentence, self.tokenizer, self.max_length)
            sentence_encodings.append(sentence_encoding)

            # Encode the SRL components
            srl_encoding = {
                'predicate': get_embedding(srl['predicate'], self.tokenizer, self.max_length),
                'ARG0': get_embedding(' '.join(srl['ARG0']), self.tokenizer, self.max_length) if srl['ARG0'] else None,
                'ARG1': get_embedding(' '.join(srl['ARG1']), self.tokenizer, self.max_length) if srl['ARG1'] else None
            }
            srl_encodings.append(srl_encoding)

        return {
            'sentence_encodings': sentence_encodings,
            'srl_encodings': srl_encodings,
            'label': torch.tensor(label, dtype=torch.float)  # Note the float type for BCEWithLogitsLoss
        }


In [134]:
# Tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [135]:
from sklearn.model_selection import train_test_split

# Assuming X, X_srl, and y are your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Similarly split the X_srl data
X_srl_train, X_srl_test = train_test_split(X_srl, test_size=0.2, random_state=42)

In [139]:
train_dataset = ArticleDataset(X_train, X_srl_train, y_train, tokenizer, max_length=128)
test_dataset = ArticleDataset(X_test, X_srl_test, y_test, tokenizer, max_length=128)

In [162]:
import pickle

# Try pickling it
try:
    serialized = pickle.dumps(test_dataset)
    print("Serialization successful!")
except Exception as e:
    print("Failed to serialize:", e)

Serialization successful!


In [164]:
path = "../notebooks/classifier/test_dataset.pkl"
with open(path, "wb") as f:
    f.write(serialized)

In [151]:
train_dataset[57]

{'sentence_encodings': [{'input_ids': [101, 8398, 6083, 2002, 2052, 2031, 3856, 2178, 12943, 2040, 2071, 2031, 8534, 3607, 15113, 2899, 1006, 13229, 1007, 2343, 6221, 8398, 4081, 9857, 2008, 2002, 2052, 2031, 3856, 2178, 4905, 2236, 2065, 2009, 2052, 2031, 3030, 2019, 4812, 2046, 3607, 19960, 14423, 1999, 1996, 2355, 2602, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
  {'input_ids': [101,

#  model

In [167]:
import torch.nn as nn
import torch.nn.functional as F

class ViewAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, K, embedding_dim):
        super(ViewAutoencoder, self).__init__()
        
        # Encoder layers
        self.Wh = nn.Linear(2 * embedding_dim, hidden_dim)  # Shared layer across views
        self.Wz = nn.Linear(hidden_dim, K)  # View-specific layer
        
        # View-specific dictionary (initialized randomly here, can be further refined)
        self.Fz = nn.Parameter(torch.randn(K, embedding_dim))
        
        # Temperature for Gumbel-Softmax
        self.tau = 1.0

    def encode(self, v_tilda):
        h = F.relu(self.Wh(v_tilda))
        lz = self.Wz(h)
        return lz

    def decode(self, lz):
        dz = F.softmax(lz, dim=1)
        
        # Gumbel-Softmax sampling
        g = torch.rand_like(dz)
        gz = F.softmax((dz.log() + g) / self.tau, dim=1)
        
        # Reconstruct view-specific span embedding
        v_hat = torch.mm(gz, self.Fz)
        return v_hat

    def forward(self, v_tilda):
        lz = self.encode(v_tilda)
        v_hat = self.decode(lz)
        return v_hat, lz

# Dummy test
autoencoder = ViewAutoencoder(input_dim=100, hidden_dim=50, K=15, embedding_dim=768)
v_tilda_example = torch.randn(10, 2 * 768)
output, _ = autoencoder(v_tilda_example)
output.shape


torch.Size([10, 768])

In [166]:
!pip install torchtext

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 96.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 MB 23.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.4 MB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 15.8 MB/